In [1]:
#imports

import numpy as np
import shelve
from astropy.table import QTable
import matplotlib.pyplot as plt
from astropy import units as u
from astropy import constants as const
from astropy.table import QTable
import os
import math
import shutil
from IPython.core.debugger import Tracer
import subprocess
import re
%matplotlib inline

In [4]:
ic = QTable.read('IC_table.csv',format='ascii.csv')
sim_idx = ic['Sim #']
dMass_b = ic['Mass_b']
dMass_c = ic['Mass_c']
dRadius_b = ic['Radius_b']
dRadius_c = ic['Radius_c']
dSemi_b = ic['Semi_b_0']
dSemi_c = ic['Semi_c_0']
dEcc_b = ic['Ecc_b_0']
dEcc_c = ic['Ecc_c_0']
dInc_b = ic['Incl_b_0']
dInc_c = ic['Incl_c_0']
dLongA_b = ic['LongA_b_0']
dLongA_c = ic['LongA_c_0']
dArgP_b = ic['ArgP_b_0']
dArgP_c = ic['ArgP_c_0']
dTidalQ_b = ic['Qp_b']
dTidalQ_c = ic['Qp_c']
dTidalQ_star = ic['Q_star']
dMass_star = ic['Mass_star']
dRadius_star = ic['Radius_star']

In [15]:
# picks out earthlike inner, circ inner, close-in inner, close-in outer
# picked 00857

selection_mask = np.where((np.abs(dMass_b)<=1) & (dEcc_b <= 0.01) & (dSemi_b <= 0.1) & (dSemi_c <= 1))
ic[857]

Sim #,Mass_b,Mass_c,Radius_b,Radius_c,Semi_b_0,Semi_c_0,Ecc_b_0,Ecc_c_0,Incl_b_0,Incl_c_0,LongA_b_0,LongA_c_0,ArgP_b_0,ArgP_c_0,Qp_b,Qp_c,Q_star,Mass_star,Radius_star
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
857.0,-0.939770900815,-16.8767782348,-0.98326146697,-4.52688840489,0.0367296251244,0.304380737188,0.00443119135111,0.158676336859,0.0,2.5903491442,106.132404193,287.909978502,342.836478148,144.298131559,88.7026249324,4289070.34084,2317458.86215,1.19191866025,0.00572120956918


In [12]:
def calc_mass_Qp(radius,toggle):
    r = radius
    R_crit = 2*u.earthRad
    if r < R_crit:
        mass = ((r/const.R_earth)**(3.68))*const.M_earth
        mass = mass.to(u.earthMass)
        Qplanet = np.random.uniform(30,301)
    else:
        r = r.to(u.cm)
        volume = (4*np.pi*r**3)/3
        density = (1*u.g)/(1*u.cm)**3
        mass = volume * density
        mass = mass.to(u.earthMass)
        Qplanet = np.random.uniform(10**6,(10**7)+1)
    if toggle == 'm':
        return radius, mass
    elif toggle == 'Q':
        return radius, Qplanet
    elif toggle == 'b' or toggle == 'both':
        return radius, mass, Qplanet
    
r = 3 * u.earthRad
calc_mass_Qp(r,'b')

(<Quantity 3.0 earthRad>,
 <Quantity 4.911950054137448 earthMass>,
 6765686.763043897)

In [6]:
'''
We need to run ~10 simulations:
- variations in the outer planet
    - sup_nep_mass/rad (6 earth rad)
        - circ
            - long
            - short
        - ecc
            - long
            - short
    - sup_earth_mass/rad (2 earth rad)
        - circ
            - long
            - short
        - ecc
            - long
            - short

'''

-48.155155168563255